In [1]:

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
 tf.enable_eager_execution()

In [3]:
path_to_file=tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


In [4]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of text: ', len(text))

Length of text:  1115394


In [5]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


In [7]:
#before training we will convert strings to numbers, and hence create twolookup tables, one mapping characters to numbers and one numbers to characters

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print('{')
for char, _ in zip(char2idx, range(20)):
    print(' {:4s}: {:3d}'.format(repr(char), char2idx[char]))
print(' ...\n')

{
 '\n':   0
 ' ' :   1
 '!' :   2
 '$' :   3
 '&' :   4
 "'" :   5
 ',' :   6
 '-' :   7
 '.' :   8
 '3' :   9
 ':' :  10
 ';' :  11
 '?' :  12
 'A' :  13
 'B' :  14
 'C' :  15
 'D' :  16
 'E' :  17
 'F' :  18
 'G' :  19
 ...



In [9]:
print('{} < --- characters mapped to int --- > {}'.format(repr(text[:13]),text_as_int[:13]))

'First Citizen' < --- characters mapped to int --- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [10]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

# i.numpy could only be called after enabling eager execution

F
i
r
s
t


In [15]:
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

for item in sequences.take(5):
     print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [20]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [21]:
for input_example, target_example in dataset.take(2):
    print(repr(''.join(idx2char[input_example.numpy()])))
    print(repr(''.join(idx2char[target_example.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you '
're all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


In [22]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5],target_example[:5])):
        print("step ", i)
        print("input ", input_idx," ", repr(idx2char[input_idx]))
        print("output ", target_idx, " ", repr(idx2char[target_idx]))

step  0
input  tf.Tensor(39, shape=(), dtype=int32)   'a'
output  tf.Tensor(56, shape=(), dtype=int32)   'r'
step  1
input  tf.Tensor(56, shape=(), dtype=int32)   'r'
output  tf.Tensor(43, shape=(), dtype=int32)   'e'
step  2
input  tf.Tensor(43, shape=(), dtype=int32)   'e'
output  tf.Tensor(1, shape=(), dtype=int32)   ' '
step  3
input  tf.Tensor(1, shape=(), dtype=int32)   ' '
output  tf.Tensor(39, shape=(), dtype=int32)   'a'
step  4
input  tf.Tensor(39, shape=(), dtype=int32)   'a'
output  tf.Tensor(50, shape=(), dtype=int32)   'l'


In [23]:
## create training batches

In [24]:
BATCH_SIZE =64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(dataset)

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>


In [25]:
#build model

vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [27]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [29]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    rnn_units= rnn_units,
    batch_size = BATCH_SIZE
)

In [30]:
import pickle

In [ ]:
pickle.dump(model,open('rnn_mode'))